In [19]:
import pandas as pd
import numpy as np
from datetime import datetime
import json, io, base64, re, os, requests, time, traceback
from IPython.display import display, HTML, clear_output

from collections import defaultdict

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

import ipywidgets as widgets
from xml.etree import ElementTree
# Initialize settings
import _settings as settings

# Global variables from settings
spec_translate_list = settings.SPEC_TRANSLATE_LIST
plotly_colors = settings.plotly_colors

In [20]:
class DataTransformation:
    def __init__(self):
        self.merged_df = None
        self.proteins_dic = {}
        self.output_area = None
        self.merged_uploader = None

    def create_download_link(self, file_path, label):
        """Create a download link for a file."""
        if os.path.exists(file_path):
            # Read file content and encode it as base64
            with open(file_path, 'rb') as f:
                content = f.read()
            b64_content = base64.b64encode(content).decode('utf-8')

            # Generate the download link HTML
            return widgets.HTML(f"""
                <a download="{os.path.basename(file_path)}" 
                   href="data:application/octet-stream;base64,{b64_content}" 
                   style="color: #0366d6; text-decoration: none; margin-left: 20px; font-size: 14px;">
                    {label}
                </a>
            """)
        else:
            # Show an error message if the file does not exist
            return widgets.HTML(f"""
                <span style="color: red; margin-left: 20px; font-size: 14px;">
                    File "{file_path}" not found!
                </span>
            """)

    def setup_data_loading_ui(self):
        """Initialize and display the data loading UI."""
        # Create file upload widget
        self.merged_uploader = widgets.FileUpload(
            accept='.csv,.txt,.tsv,.xlsx',
            multiple=False,
            description='Upload Merged Data File',
            layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )

        self.output_area = widgets.Output()

        # Create upload box with example link
        merged_box = widgets.HBox([
            self.merged_uploader,
            self.create_download_link("example_merged_dataframe.csv", "Example")
        ], layout=widgets.Layout(align_items='center'))

        # Create left column with upload widgets
        upload_widgets = widgets.VBox([
            widgets.HTML("<h4>Upload Data File:</h4>"),
            merged_box,
            self.output_area
        ], layout=widgets.Layout(
            width='300px',
            margin='0 20px 0 0'
        ))

        # Create container for status display
        self.status_area = widgets.Output(
            layout=widgets.Layout(
                width='300px',
                margin='0 0 0 20px'
            )
        )

        display(upload_widgets,
                self.status_area)

        # Register observer
        self.merged_uploader.observe(self._on_merged_upload_change, names='value')

    def _validate_and_clean_data(self, df):
        """
        Validate and clean the uploaded data, preserving numeric data even if stored as strings.
        Returns tuple of (cleaned_df, warnings, errors)
        """
        warnings = []
        errors = []
        cleaned_df = df.copy()
    
        # Check required columns exist
        required_columns = [
            'Master Protein Accessions', 
            'unique ID'
        ]
        
        # Check that at least one Avg_ column exists
        avg_columns = [col for col in df.columns if col.startswith('Avg_')]
        if not avg_columns:
            errors.append("No columns starting with 'Avg_' found in the data")
            return None, warnings, errors
            
        # Add Avg_ columns to required columns
        required_columns.extend(avg_columns)
        
        missing = set(required_columns) - set(df.columns)
        if missing:
            errors.append(f"Missing required columns: {', '.join(missing)}")
            return None, warnings, errors
    
        # Separate numeric and non-numeric columns
        numeric_columns = avg_columns  # Avg_ columns should be numeric
        text_columns = ['Master Protein Accessions', 'unique ID']
    
        # Handle blank values differently for numeric vs text columns
        for column in required_columns:
            if column in numeric_columns:
                # For numeric columns, try to convert to numeric first
                try:
                    # Convert to numeric, coerce errors to NaN
                    cleaned_df[column] = pd.to_numeric(cleaned_df[column], errors='coerce')
                    blank_count = cleaned_df[column].isna().sum()
                    if blank_count > 0:
                        warnings.append(f"Found {blank_count} invalid/blank numeric values in {column} column")
                except Exception as e:
                    errors.append(f"Error converting {column} to numeric: {str(e)}")
                    return None, warnings, errors
            elif column in text_columns:
                # For text columns, check for truly empty values
                blank_mask = cleaned_df[column].isna() | (cleaned_df[column].astype(str).str.strip() == '')
                blank_count = blank_mask.sum()
                if blank_count > 0:
                    warnings.append(f"Dropping {blank_count} rows with blank values in {column} column")
                    cleaned_df = cleaned_df[~blank_mask]
    
        # Check for invalid characters in non-blank rows
        if len(cleaned_df) > 0:
            # Check Positions in Proteins
            invalid_pos = cleaned_df['Positions in Proteins'].apply(
                lambda x: ',' in str(x) or ':' in str(x)
            )
            if invalid_pos.any():
                errors.append(
                    "Found invalid characters (',' or ':') in Positions in Proteins column. "
                    "Please update the file and upload again."
                )
            
            # Check Master Protein Accessions
            invalid_acc = cleaned_df['Master Protein Accessions'].apply(
                lambda x: ',' in str(x) or ':' in str(x)
            )
            if invalid_acc.any():
                errors.append(
                    "Found invalid characters (',' or ':') in Master Protein Accessions column. "
                    "Please update the file and upload again."
                )
    
        return cleaned_df, warnings, errors

    def _process_protein_info(self, df):
        """
        Process protein information from the dataframe and store in proteins_dic.
        Asks user whether to fetch from UniProt or use accession IDs when protein info is missing.
        """
        # Initialize a cache for UniProt information to avoid redundant queries
        self.uniprot_cache = getattr(self, 'uniprot_cache', {})
        
        # Check if we need to fetch any data from UniProt
        has_protein_info = all(col in df.columns for col in ['protein_name', 'protein_species'])
        if has_protein_info:
            # Check if we have valid data for all entries
            all_data_present = (
                df['protein_name'].notna().all() and 
                df['protein_species'].notna().all() and
                (df['protein_name'] != '').all() and
                (df['protein_species'] != '').all()
            )
            if all_data_present:
                # If we have all data, just process it silently
                protein_info = df.groupby('Master Protein Accessions').agg({
                    'protein_name': 'first',
                    'protein_species': 'first'
                }).reset_index()
                
                for _, row in protein_info.iterrows():
                    protein_id = row['Master Protein Accessions']
                    self.proteins_dic[protein_id] = {
                        "name": row['protein_name'],
                        "species": row['protein_species']
                    }
                return len(self.proteins_dic)

        # Store the dataframe for later processing
        self._protein_df_to_process = df
        
        # Create a flag to track if processing is complete
        self._protein_processing_complete = False
        
        # If we need to fetch data, ask the user what they want to do
        # Display in the status area
        with self.status_area:
            self.status_area.clear_output()
            
            # Create buttons for user choice
            fetch_button = widgets.Button(
                description='Query UniProt',
                button_style='info',
                tooltip='Fetch protein names from UniProt database (may take time)',
                layout=widgets.Layout(width='250px')
            )
            
            use_accession_button = widgets.Button(
                description='Use Protein IDs',
                button_style='warning',
                tooltip='Use protein accession IDs as names without querying UniProt',
                layout=widgets.Layout(width='250px')
            )
            
            # Define button click handlers
            fetch_button.on_click(lambda b: self._process_proteins_with_choice(True))
            use_accession_button.on_click(lambda b: self._process_proteins_with_choice(False))
            
            display(HTML("""
                <div style="padding: 15px; margin: 10px 0; border-left: 4px solid #17a2b8; background-color: #f8f9fa;">
                    <h4 style="margin-top: 0;">Protein Information Missing</h4>
                    <p>Some protein names or species information is missing in your data.</p>
                    <p>Would you like to:</p>
                         <ul>
                                <li>Fetch protein names from UniProt database (may take time)</li>
                                <li>Use protein accession IDs as names without querying UniProt</li>
                        </ul>
                </div>
            """))
            display(widgets.HBox([fetch_button, use_accession_button]))
        
        # Return the current count, but processing will continue when a button is clicked
        return len(self.proteins_dic)

    def _process_proteins_with_choice(self, fetch_from_uniprot):
        """
        Process proteins based on user choice.
        This is called when the user clicks one of the choice buttons.
        """
        # Get the dataframe to process
        df = self._protein_df_to_process
        
        # Clear the status area and show processing message
        with self.status_area:
            self.status_area.clear_output()
            if fetch_from_uniprot:
                display(HTML('<div style="color: #17a2b8; padding: 10px; margin: 10px 0;">Fetching protein information from UniProt...</div>'))
            else:
                display(HTML('<div style="color: #ffc107; padding: 10px; margin: 10px 0;">Using protein accession IDs as names...</div>'))
        
        # Process proteins based on user choice
        # Use the status area for progress display
        with self.status_area:
            # Initialize counters
            total_proteins = 0
            uniprot_found = 0
            uniprot_not_found = 0
            multiple_entries = 0
            cached_proteins = 0
            
            # Check if we need to fetch any data from UniProt
            has_protein_info = all(col in df.columns for col in ['protein_name', 'protein_species'])
            
            # Group by protein accession to get unique proteins
            protein_info = df.groupby('Master Protein Accessions').agg({
                'protein_name': 'first' if 'protein_name' in df.columns else lambda x: None,
                'protein_species': 'first' if 'protein_species' in df.columns else lambda x: None
            }).reset_index()

            progress_html = """
                <style>
                    .fetch-status { font-family: monospace; margin: 10px 0; padding: 10px; }
                    .fetch-progress { margin: 5px 0; padding: 5px; }
                    .success { color: #28a745; }
                    .warning { color: #ffc107; }
                    .error { color: #dc3545; }
                    .info { color: #17a2b8; }
                    .summary { margin-top: 10px; padding: 10px;}
                </style>
                <div class="fetch-status">
                    <div id="progress-updates"></div>
                </div>
            """
            
            # First, collect all proteins that need fetching
            proteins_to_fetch = []
            
            for _, row in protein_info.iterrows():
                total_proteins += 1
                protein_id = row['Master Protein Accessions']
                
                # Skip entries with multiple protein IDs
                if ';' in protein_id:
                    multiple_entries += 1
                    self.proteins_dic[protein_id] = {
                        "name": protein_id,
                        "species": "Multiple"
                    }
                    continue
                
                # Use existing data if available and not empty
                if (has_protein_info and 
                    pd.notna(row['protein_name']) and 
                    pd.notna(row['protein_species']) and 
                    row['protein_name'] != '' and 
                    row['protein_species'] != ''):
                    self.proteins_dic[protein_id] = {
                        "name": row['protein_name'],
                        "species": row['protein_species']
                    }
                    continue
                
                # Check if we already have this protein in cache
                if protein_id in self.uniprot_cache:
                    cached_proteins += 1
                    name, species = self.uniprot_cache[protein_id]
                    self.proteins_dic[protein_id] = {
                        "name": name,
                        "species": species
                    }
                    continue
                
                # If we need to fetch and user chose to fetch from UniProt
                if fetch_from_uniprot:
                    proteins_to_fetch.append(protein_id)
                else:
                    # Use accession ID as name
                    self.proteins_dic[protein_id] = {
                        "name": protein_id,
                        "species": "Unknown"
                    }
            
            # Process proteins in batches if fetching from UniProt
            if fetch_from_uniprot and proteins_to_fetch:
                # Update progress display
                display(HTML(progress_html + f"""
                    <div class="fetch-progress info">
                        Preparing to fetch {len(proteins_to_fetch)} proteins from UniProt in batches...
                    </div>
                """))
                
                # Process in batches of 50 (adjust as needed)
                batch_size = 50
                total_batches = (len(proteins_to_fetch) + batch_size - 1) // batch_size
                
                for batch_num in range(total_batches):
                    start_idx = batch_num * batch_size
                    end_idx = min((batch_num + 1) * batch_size, len(proteins_to_fetch))
                    current_batch = proteins_to_fetch[start_idx:end_idx]
                    
                    # Update progress
                    self.status_area.clear_output(wait=True)
                    display(HTML(progress_html + f"""
                        <div class="fetch-progress info">
                            Fetching batch {batch_num + 1}/{total_batches} ({len(current_batch)} proteins)...
                        </div>
                        <div class="summary">
                            <h4>Progress:</h4>
                            <ul>
                                <li>Total proteins: {total_proteins}</li>
                                <li>Proteins from cache: {cached_proteins}</li>
                                <li>UniProt matches found: {uniprot_found}</li>
                                <li>UniProt matches not found: {uniprot_not_found}</li>
                                <li>Multiple entry proteins: {multiple_entries}</li>
                                <li>Remaining to fetch: {len(proteins_to_fetch) - start_idx}</li>
                            </ul>
                        </div>
                    """))
                    
                    # Fetch the batch using individual queries for now
                    # We'll implement batch fetching later
                    batch_results = {}
                    for protein_id in current_batch:
                        try:
                            name, species = self.fetch_uniprot_info(protein_id)
                            if name and species:
                                batch_results[protein_id] = (name, species)
                        except Exception as e:
                            print(f"Error fetching {protein_id}: {str(e)}")
                    
                    # Process the results
                    for protein_id in current_batch:
                        if protein_id in batch_results:
                            name, species = batch_results[protein_id]
                            uniprot_found += 1
                            self.proteins_dic[protein_id] = {
                                "name": name,
                                "species": species
                            }
                            # Add to cache
                            self.uniprot_cache[protein_id] = (name, species)
                        else:
                            uniprot_not_found += 1
                            self.proteins_dic[protein_id] = {
                                "name": protein_id,
                                "species": "Unknown"
                            }
                            # Cache the negative result too
                            self.uniprot_cache[protein_id] = (protein_id, "Unknown")
            
            # Show final summary
            self.status_area.clear_output(wait=True)
            display(HTML(f"""
                <div class="fetch-status">
                    <h4 style="color:green;"><b>Protein Processing Complete!</b></h4>
                    <div class="summary">
                        <h4>Final Summary:</h4>
                        <ul>
                            <li>Total proteins processed: {total_proteins}</li>
                            <li>Proteins from cache: {cached_proteins}</li>
                            <li>Multiple entry proteins: {multiple_entries}</li>
                            {"<li>UniProt matches found: " + str(uniprot_found) + "</li>" if fetch_from_uniprot else ""}
                            {"<li>UniProt matches not found: " + str(uniprot_not_found) + "</li>" if fetch_from_uniprot else ""}
                        </ul>
                    </div>
                </div>
            """))
        
        # Mark processing as complete
        self._protein_processing_complete = True
        
        # Update the merged_df with the protein information
        if hasattr(self, 'merged_df') and self.merged_df is not None:
            # Add protein_name and protein_species columns if they don't exist
            if 'protein_name' not in self.merged_df.columns:
                self.merged_df['protein_name'] = ''
            if 'protein_species' not in self.merged_df.columns:
                self.merged_df['protein_species'] = ''
            
            # Update the columns with the fetched information
            for protein_id, info in self.proteins_dic.items():
                mask = self.merged_df['Master Protein Accessions'] == protein_id
                self.merged_df.loc[mask, 'protein_name'] = info['name']
                self.merged_df.loc[mask, 'protein_species'] = info['species']
        
        # Return the number of proteins processed
        return len(self.proteins_dic)

    def fetch_uniprot_info_batch(self, protein_ids, max_retries=3, timeout=30):
        """
        Fetch protein information for multiple proteins at once using UniProt's batch API.
        Returns a dictionary mapping protein IDs to (name, species) tuples.
        """
        if not protein_ids:
            return {}
        
        results = {}
        
        try:
            # Use the batch REST API endpoint
            batch_url = 'https://rest.uniprot.org/uniprotkb/search'
            
            # Create a query with OR conditions for each protein ID
            query = ' OR '.join([f'accession:{pid}' for pid in protein_ids])
            
            # Parameters for the request
            params = {
                'query': query,
                'format': 'json',
                'fields': 'accession,protein_name,organism_name',
                'size': len(protein_ids)  # Request all results in one response
            }
            
            # Make the request with timeout and retry logic
            retry_count = 0
            while retry_count < max_retries:
                try:
                    response = requests.get(batch_url, params=params, timeout=timeout)
                    
                    # Handle rate limiting
                    if response.status_code == 429:  # Too Many Requests
                        retry_after = int(response.headers.get('Retry-After', 5))
                        print(f"Rate limited. Waiting {retry_after} seconds...")
                        time.sleep(retry_after + 1)  # Add 1 second buffer
                        retry_count += 1
                        continue
                    
                    # Break if successful
                    if response.status_code == 200:
                        break
                    
                    # Handle other errors
                    response.raise_for_status()
                    
                except (requests.exceptions.Timeout, requests.exceptions.ConnectionError) as e:
                    retry_count += 1
                    wait_time = 2 ** retry_count  # Exponential backoff
                    print(f"Request failed: {str(e)}. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
            
            # Process the response
            if response.status_code == 200:
                data = response.json()
                
                for entry in data.get('results', []):
                    accession = entry.get('primaryAccession')
                    
                    # Extract protein name
                    protein_name = None
                    protein_data = entry.get('proteinDescription', {})
                    
                    # Try to find a common/short name first
                    if 'recommendedName' in protein_data:
                        if 'shortNames' in protein_data['recommendedName']:
                            protein_name = protein_data['recommendedName']['shortNames'][0]['value']
                        else:
                            protein_name = protein_data['recommendedName'].get('fullName', {}).get('value')
                    
                    # Try alternative names if no recommended name found
                    if not protein_name and 'alternativeNames' in protein_data and protein_data['alternativeNames']:
                        if 'shortNames' in protein_data['alternativeNames'][0]:
                            protein_name = protein_data['alternativeNames'][0]['shortNames'][0]['value']
                        else:
                            protein_name = protein_data['alternativeNames'][0].get('fullName', {}).get('value')
                    
                    # Extract species name
                    species = None
                    organism_data = entry.get('organism', {})
                    organism_names = organism_data.get('names', [])
                    
                    # Try to find common name first
                    for name in organism_names:
                        if name['type'] == 'common':
                            species = name['value']
                            break
                    
                    # Fallback to scientific name
                    if not species and organism_names:
                        for name in organism_names:
                            if name['type'] == 'scientific':
                                species = name['value']
                                break
                    
                    # Clean up protein name if found
                    if protein_name:
                        protein_name = protein_name.split(' precursor')[0].split(' (')[0]
                    
                    # Store the results
                    if accession and (protein_name or species):
                        results[accession] = (protein_name or accession, species or "Unknown")
            
            return results
        
        except Exception as e:
            print(f"Error in batch fetch: {str(e)}")
            return {}
        
    def _load_merged_data(self, file_data):
        """
        Load and validate merged data file
        Returns tuple of (dataframe, status)
        """
        try:
            content = bytes(file_data.content)
            filename = file_data.name
            extension = filename.split('.')[-1].lower()

            file_stream = io.BytesIO(content)

            # Load data based on file extension
            try:
                if extension == 'csv':
                    df = pd.read_csv(file_stream)
                elif extension in ['txt', 'tsv']:
                    df = pd.read_csv(file_stream, delimiter='\t')
                elif extension == 'xlsx':
                    df = pd.read_excel(file_stream)
                else:
                    display(HTML(f'<b style="color:red;">Error: Unsupported file format</b>'))
                    return None, 'no'
            except Exception as e:
                display(HTML(f'<b style="color:red;">Error reading file: {str(e)}</b>'))
                return None, 'no'

            # Check for protein info columns and notify user
            missing_columns = []
            if 'protein_name' not in df.columns:
                missing_columns.append('protein_name')
                df['protein_name'] = ''
            if 'protein_species' not in df.columns:
                missing_columns.append('protein_species')
                df['protein_species'] = ''
                
            if missing_columns:
                notification = f"""
                <div style="padding: 10px; margin: 10px 0;">
                    <p style="color: #17a2b8; margin: 0;">
                        <b>Notice:</b> The following columns are missing from your data:
                        <ul style="color: #17a2b8; margin: 5px 0;">
                            {''.join(f'<li>{col}</li>' for col in missing_columns)}
                        </ul>
                        </p>
                        <p style="color: #17a2b8; margin: 0;">
                        UniProt will be searched to automatically fill in this information. <br>
                        Alternativly you can upload a standardized file from the data transomation module with the protein information. 
                    </p>
                </div>
                """
                display(HTML(notification))

            # Validate and clean data
            cleaned_df, warnings, errors = self._validate_and_clean_data(df)

            # Warnings about invalid/blank values are commented out
            # if warnings:
            #     warning_html = "<br>".join([
            #         f'<b style="color:orange;">Warning: {w}</b>'
            #         for w in warnings
            #     ])
            #     display(HTML(warning_html))

            # Display errors if any
            if errors:
                error_html = "<br>".join([
                    f'<b style="color:red;">Error: {e}</b>'
                    for e in errors
                ])
                display(HTML(error_html))
                return None, 'no'

            if cleaned_df is not None and len(cleaned_df) > 0:
                # Process protein information
                num_proteins = self._process_protein_info(cleaned_df)
                
                # Add information about remaining rows and processed proteins
                success_message = f"""
                <div style="padding: 10px; margin: 10px 0; border-left: 4px solid #28a745; background-color: #f8f9fa;">
                    <p style="color: #28a745; margin: 0;">
                        <b>Data Import Complete!</b><br>
                        • Data imported successfully with {cleaned_df.shape[0]} rows and {cleaned_df.shape[1]} columns.<br>
                        • Processed data contains {len(cleaned_df)} rows after removing blank values.<br>
                        • Successfully processed information for {num_proteins} unique proteins.
                    </p>
                </div>
                """
                #display(HTML(success_message))
                return cleaned_df, 'yes'
            else:
                display(HTML('<b style="color:red;">Error: No valid data rows remaining after cleaning</b>'))
                return None, 'no'

        except Exception as e:
            display(HTML(f'<b style="color:red;">Error processing file: {str(e)}</b>'))
            return None, 'no'

    def _on_merged_upload_change(self, change):
        """Handle merged data file upload"""
        if change['type'] == 'change' and change['name'] == 'value':
            with self.output_area:
                self.output_area.clear_output()
                if change['new'] and len(change['new']) > 0:
                    file_data = change['new'][0]
                    df, status = self._load_merged_data(file_data)
                    if status == 'yes' and df is not None:
                        self.merged_df = df  # Only set merged_df if validation passed
                        display(HTML(
                            f'<b style="color:green;">Data imported successfully with '
                            f'{df.shape[0]} rows and {df.shape[1]} columns.</b>'
                        ))

    def fetch_uniprot_info(self, protein_id):
        """
        Fetch protein information from UniProt, prioritizing common names.
        Returns tuple of (protein_common_name, species_common_name) or (None, None) if not found.
        """
        try:
            # Try REST API first
            rest_url = f'https://rest.uniprot.org/uniprotkb/{protein_id}.json'
            response = requests.get(rest_url)
            
            if response.status_code == 200:
                data = response.json()
                
                # Get protein common name
                try:
                    # Look for protein names
                    names = data['proteinDescription']
                    protein_name = None
                    
                    # Try to find a common/short name
                    if 'shortNames' in names.get('recommendedName', {}):
                        protein_name = names['recommendedName']['shortNames'][0]['value']
                    elif 'shortNames' in names.get('alternativeNames', [{}])[0]:
                        protein_name = names['alternativeNames'][0]['shortNames'][0]['value']
                    else:
                        # Fallback to full name
                        protein_name = names.get('recommendedName', {}).get('fullName', {}).get('value')
                    
                    # Get species common name
                    organism_data = data['organism']
                    species = None
                    for name in organism_data.get('names', []):
                        if name['type'] == 'common':
                            species = name['value']
                            break
                    if not species:  # Fallback to scientific name
                        species = organism_data.get('scientificName')
                    
                    return protein_name, species
                    
                except KeyError:
                    pass  # Fall through to XML approach
            
            # Fall back to XML API
            xml_url = f'https://www.uniprot.org/uniprot/{protein_id}.xml'
            response = requests.get(xml_url)
            
            if response.status_code != 200:
                return None, None
                
            root = ElementTree.fromstring(response.content)
            ns = {'up': 'http://uniprot.org/uniprot'}
            
            # Get protein common name with fallbacks
            protein_name = None
            # Try short name first
            name_element = (
                root.find('.//up:recommendedName/up:shortName', ns) or
                root.find('.//up:alternativeName/up:shortName', ns) or
                root.find('.//up:recommendedName/up:fullName', ns) or
                root.find('.//up:submittedName/up:fullName', ns)
            )
            protein_name = name_element.text if name_element is not None else None
            
            # Get species common name
            species = None
            # Try common name first
            organism = root.find('.//up:organism/up:name[@type="common"]', ns)
            if organism is not None:
                species = organism.text
            else:
                # Fallback to scientific name
                organism = root.find('.//up:organism/up:name[@type="scientific"]', ns)
                species = organism.text if organism is not None else "Unknown"
            
            if protein_name:
                # Clean up protein name - remove any "precursor" or similar suffixes
                protein_name = protein_name.split(' precursor')[0].split(' (')[0]
                return protein_name, species
            else:
                return None, None
            
        except Exception as e:
            print(f"Error fetching UniProt data for {protein_id}: {str(e)}")
            return None, None

In [21]:
class PlotState:
    def __init__(self):
        self.current_state = {
            'uploadedData': None,
            'topProteins': None,
            'groupSelection': None,
            'xLabel': None,
            'yLabel': None,
            'colorScheme': None,
            'lastGenerated': None,
            'buttonsLocked': True
        }
    
    def update_state(self, **kwargs):
        self.current_state.update(kwargs)
        # Lock buttons when state changes
        self.current_state['buttonsLocked'] = True
    
    def generate_completed(self):
        self.current_state['buttonsLocked'] = False
        self.current_state['lastGenerated'] = {
            key: value for key, value in self.current_state.items() 
            if key not in ['lastGenerated', 'buttonsLocked']
        }
    
    def get_state(self):
        return self.current_state

In [22]:
 
class ProteinPlotter:
    def __init__(self, data_transformer):
        self.data_transformer = data_transformer
        self.plot_output = widgets.Output()
        self.info_output = widgets.Output()
        self.export_output = widgets.Output()
        self.proteins_df = None
        self.sum_df = None
        
        # Initialize state manager first
        self.state_manager = PlotState()

        self.download_plot_button = widgets.Button(
            description='Download Interactive Plot',
            button_style='info',
            icon='file',
            layout=widgets.Layout(width='200px')
        )

        # Create multi-select widget for groups
        self.group_select = widgets.SelectMultiple(
            options=[],
            description='Groups:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='100px')
        )
        
        self.plot_button = widgets.Button(
            description='Generate/Update Data',
            button_style='success',
            icon='refresh',
            layout=widgets.Layout(width='200px')
        )
        
        self.export_button = widgets.Button(
            description='Export Data',
            button_style='info',
            icon='download',
            layout=widgets.Layout(width='200px')
        )

        # Add label customization widgets
        self.xlabel_widget = widgets.Text(
            description='X Label:',
            placeholder='Enter x-axis label',
            layout=widgets.Layout(width='300px')
        )
        
        self.ylabel_widget = widgets.Text(
            description='Y Label:',
            placeholder='Enter y-axis label',
            layout=widgets.Layout(width='300px')
        )
        self.legend_widget = widgets.Text(
            description='Legend Title',
            placeholder='Enter a custom legend title',
            layout=widgets.Layout(width='300px')
        )
        self.title_widget = widgets.Text(
            description='Plot Title',
            placeholder='Enter a custom plot title',
            layout=widgets.Layout(width='300px')
        )


        # Update color scheme dropdown with categorized options
        color_schemes = [
            '--- DEFAULT (HSV)---',
            'HSV',  # Default option
            '--- QUALITATIVE (BEST FOR CATEGORIES) ---',
            'Plotly', 'D3', 'G10', 'T10', 'Alphabet', 
            'Set1', 'Set2', 'Set3', 'Pastel1', 'Pastel2', 'Paired',
            '--- SEQUENTIAL ---',
            'Viridis', 'Cividis', 'Inferno', 'Magma', 'Plasma',
            'Hot', 'Jet', 'Blues', 'Greens', 'Reds', 'Purples', 'Oranges',
            '--- DIVERGING ---',
            'Spectral', 'RdBu', 'RdYlBu', 'RdYlGn', 'PiYG', 'PRGn', 'BrBG', 'RdGy',
            '--- CYCLICAL ---',
            'IceFire', 'Edge', 'Twilight'
        ]
        
        self.color_scheme = widgets.Dropdown(
            options=color_schemes,
            value='HSV',  # Default value
            description='Color Scheme:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )

        # Add an inversion toggle radio button
        self.invert_plot = widgets.RadioButtons(
            description='Plot Orientation:',
            options=['By Sample', 'By Protein'],
            value='By Sample',  # Default selection
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='auto'),
            disabled=False,
            indent=True  # Keeps options aligned with description instead of appearing below
        )

        # Change the protein_selector initialization to not have a default selection
        self.protein_selector = widgets.SelectMultiple(
            options=[],
            value=(),  # Empty tuple - no selection by default
            description='Proteins:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(
                width='300px',
                height='100px',
            )
        )
        # Selecte between relative and absolute plots
        self.metric_type = widgets.RadioButtons(
            description='Scale Absorbance:',
            options=['Relative', 'Absolute'],
            value='Relative',  # Default selection
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='auto'),
            disabled=False,
            indent=True  # Keeps options aligned with description instead of appearing below
        )                   
        # Create the checkbox with improved description
        self.plot_minor_proteins = widgets.Checkbox(
            description='Show Minor Proteins',
            value=True,
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px', height='30px')
        )

        # Use the existing create_help_icon function style
        def create_help_icon(tooltip_text):
            """Create a help icon widget with tooltip"""
            help_icon = widgets.HTML(
                value='<i class="fa fa-question-circle" style="color: #007bff;"></i>',
                layout=widgets.Layout(width='25px', margin='2px 5px')
            )
            help_icon.add_class('jupyter-widgets')
            help_icon.add_class('widget-html')
            return widgets.HTML(
                f'<div title="{tooltip_text}" style="display: inline-block;">{help_icon.value}</div>'
            )

        # Create a help icon with explanatory tooltip
        help_tooltip = "Groups all unselected proteins into a single 'Minor Proteins' category in the plot"
        minor_proteins_help = create_help_icon(help_tooltip)

        # Combine checkbox and help icon into a horizontal layout
        self.minor_proteins_row = widgets.HBox([
            self.plot_minor_proteins, 
            minor_proteins_help
        ], layout=widgets.Layout(align_items='center'))

        # Add after creating the protein_selector widget
        self._populate_protein_selector()


        # Update plot type selection to remove 'All Plots'
        self.plot_type = widgets.RadioButtons(
            options=['Stacked Bar Plots', 'Grouped Bar Plots', 'Pie Charts'],
            value='Stacked Bar Plots',
            description='Plot Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )
        

        # Add bar plot type selection
        self.abs_or_count = widgets.RadioButtons(
            options=['Absorbance', 'Count'],
            value='Absorbance', 
            description='Data Type:',
            style={'description_width': 'initial'},
            layout=widgets.Layout(width='300px')
        )

        # Create help icons
        def create_help_icon(tooltip_text):
            """Create a help icon widget with tooltip"""
            help_icon = widgets.HTML(
                value='<i class="fa fa-question-circle" style="color: #007bff;"></i>',
                layout=widgets.Layout(width='25px', margin='2px 5px')
            )
            help_icon.add_class('jupyter-widgets')
            help_icon.add_class('widget-html')
            return widgets.HTML(
                f'<div title="{tooltip_text}" style="display: inline-block;">{help_icon.value}</div>'
            )

        # Add help tooltips
        plot_type_help = create_help_icon("Select whether to display data as a bar plot or pie chart")
        bar_plot_type_help = create_help_icon("Choose the type of values to display in the bar plot")
        plot_orientation_help = create_help_icon("Group data by sample or by protein")

        self.plot_type_row = widgets.HBox([self.plot_type,
                                           self.metric_type],
                                           layout=widgets.Layout(width='300px')
                                          )
        self.plot_type_row_two= widgets.HBox([self.abs_or_count,
                                           self.invert_plot],
                                           layout=widgets.Layout(width='300px')
                                          )
        # Create layout
        self.widget_box = widgets.VBox([
            widgets.HTML("<h4>Plot Controls:</h4>"),
            self.group_select,
            self.protein_selector,
            self.plot_type_row,
            self.plot_type_row_two,
            self.minor_proteins_row,
            widgets.HTML("<h4>Appearance Settings:</h4>"),
            self.xlabel_widget,
            self.ylabel_widget,
            self.legend_widget,
            self.title_widget,
            self.color_scheme,
            widgets.HTML("<h4>Actions:</h4>"),
            self.plot_button,
            self.export_button,
            self.download_plot_button,
            self.info_output,
            self.plot_output,
            self.export_output
        ])
            
            
        # Set initial button states
        self.export_button.disabled = True
        self.download_plot_button.disabled = True
        
        def _on_input_change(change):
            self.state_manager.update_state()
            self.export_button.disabled = True
            self.download_plot_button.disabled = True

        
        # Add observers for input changes
        self.group_select.observe(_on_input_change, names='value')
        self.xlabel_widget.observe(_on_input_change, names='value')
        self.ylabel_widget.observe(_on_input_change, names='value')
        self.legend_widget.observe(_on_input_change, names='value')
        self.title_widget.observe(_on_input_change, names='value')
        self.color_scheme.observe(_on_input_change, names='value')

        # Add button click handlers
        self.plot_button.on_click(self._on_plot_button_click)
        self.export_button.on_click(self._on_export_button_click)
        self.download_plot_button.on_click(self._on_download_plot_click)
        
        # Add variable to store current figure
        self.current_fig = None
        # Add observer for data changes
        self.data_transformer.merged_uploader.observe(self._update_group_options, names='value')

        # Register an explicit callback to populate proteins when merged data changes
        self.data_transformer.merged_uploader.observe(self._populate_protein_selector, names='value')
        
        # In your setup_widgets method, add this line:
        self.color_scheme.observe(self._on_color_scheme_change, names='value')
  
    def _update_group_options(self, change):
        """Update group selection options when data changes"""
        if self.data_transformer.merged_df is not None:
            # Get all Avg_ columns
            avg_columns = [col.replace('Avg_', '') for col in self.data_transformer.merged_df.columns 
                         if col.startswith('Avg_')]
            
            # Update group selection options
            self.group_select.options = avg_columns
            # Select all groups by default
            self.group_select.value = avg_columns

    def _populate_protein_selector(self, change=None):
        """Populate the protein selector with proteins ordered by their relative abundance across all samples"""
        
        # Check if data_transformer is available
        if not hasattr(self, 'data_transformer') or self.data_transformer is None:
            return
            
        # Use proteins_dic (with 's') instead of protein_dic
        if not hasattr(self.data_transformer, 'proteins_dic') or not self.data_transformer.proteins_dic:
            return
            
        try:
            # Calculate protein abundance across all samples
            protein_abundance = {}
            
            if hasattr(self.data_transformer, 'merged_df') and self.data_transformer.merged_df is not None:
                df = self.data_transformer.merged_df
                
                # Find all Avg_ columns for abundance data
                abundance_cols = [col for col in df.columns if col.startswith('Avg_')]
                protein_col = 'Master Protein Accessions'
                
                if abundance_cols and protein_col in df.columns:
                    
                    # Process each row in the dataframe
                    for _, row in df.iterrows():
                        # Skip rows without protein information
                        if pd.isna(row[protein_col]) or row[protein_col] == '':
                            continue
                            
                        # Get proteins for this peptide
                        proteins = [p.strip() for p in str(row[protein_col]).split(';') if p.strip()]
                        
                        # Calculate total abundance across all samples for this peptide
                        total_abundance = 0
                        for col in abundance_cols:
                            try:
                                if pd.notna(row.get(col)):
                                    total_abundance += float(row.get(col, 0))
                            except (ValueError, TypeError) as e:
                                print(f"Error converting abundance value in column {col}: {str(e)}")
                                print(f"Value: {row.get(col)}, Type: {type(row.get(col))}")
                        
                        # If there are multiple proteins, divide the abundance equally among them
                        per_protein_abundance = total_abundance / len(proteins) if proteins else 0
                        
                        # Add to each protein's total
                        for protein in proteins:
                            if protein in protein_abundance:
                                protein_abundance[protein] += per_protein_abundance
                            else:
                                protein_abundance[protein] = per_protein_abundance
                    
            
            # Get the list of all proteins from proteins_dic
            all_proteins = list(self.data_transformer.proteins_dic.keys())
            
            # Sort proteins by abundance (highest first)
            if protein_abundance:
                # Get proteins sorted by abundance
                sorted_proteins = sorted(all_proteins, 
                                        key=lambda p: protein_abundance.get(p, 0), 
                                        reverse=True)

                # Create options with protein ID and name
                options = []
                options.append('All')  # Add 'All' option first
                
                # Add each protein with its ID, name and abundance
                for protein_id in sorted_proteins:
                    protein_info = self.data_transformer.proteins_dic.get(protein_id, {})
                    protein_name = protein_info.get('name', protein_id)
                    abundance = protein_abundance.get(protein_id, 0)

                    options.append(protein_name)
                    
                # Update the protein selector with all options
                self.protein_selector.options = options
                
                # Automatically select the top 10 proteins (or fewer if less are available)
                num_proteins = min(10, len(options) - 1)  # -1 to account for 'All'
                if num_proteins > 0:
                    # Select top proteins (options[1:] to skip 'All')
                    self.protein_selector.value = tuple(options[1:num_proteins+1])
        
        except Exception as e:
            print(f"Error populating protein selector: {str(e)}")
            import traceback
            traceback.print_exc()

    def _get_proteins_to_plot(self):
        """Get the list of proteins to plot based on user selection"""
        try:
            # 'Select Specific Proteins'
            # Get selection from multi-select widget
            if hasattr(self, 'protein_selector'):
                selected = self.protein_selector.value
                
                # Handle 'All' selection
                if 'All' in selected:
                    # Use all proteins in the dataframe
                    if hasattr(self, 'proteins_df'):
                        self.pro_list = list(set(self.proteins_df['Description']))
                    else:
                        self.pro_list = []
                    return self.pro_list
                else:
                    # Use selected proteins
                    self.pro_list = list(selected)
                    return self.pro_list
            else:
                print("Protein selector widget not found")
                self.pro_list = []
                return []
                    
        except Exception as e:
            print(f"Error getting proteins to plot: {str(e)}")
            import traceback
            traceback.print_exc()
            
            # Set empty list on error for compatibility
            self.pro_list = []
            return []

    def _on_color_scheme_change(self, change):
        """Update plot when color scheme changes"""
        if self.current_fig is not None and hasattr(self, 'plot_button'):
            # Trigger plot update by simulating a button click
            self._on_plot_button_click(None)

    def _get_avg_columns(self):
        """Get all columns that start with 'Avg_' from the merged dataframe"""
        if self.data_transformer.merged_df is not None:
            return [col for col in self.data_transformer.merged_df.columns if col.startswith('Avg_')]
        return []

    def _update_group_options(self, change):
        """Update group selection options when data changes"""
        if self.data_transformer.merged_df is not None:
            avg_columns = self._get_avg_columns()
            # Remove 'Avg_' prefix for display
            group_options = [col.replace('Avg_', '') for col in avg_columns]
            self.group_select.options = group_options
            # Select all groups by default
            self.group_select.value = group_options
                    
    def process_data(self, selected_groups=None):
        from collections import defaultdict
        
        if self.data_transformer.merged_df is None or not self.data_transformer.proteins_dic:
            return False

        # First, update the protein list to plot
        self._get_proteins_to_plot()
        
        df = self.data_transformer.merged_df.copy()
        self.merged_df = df.copy()  # Store a reference to merged_df for later use
        
        # Get Absorbance columns based on selected groups
        if selected_groups:
            Absorbance_cols = [f'Avg_{var}' for var in selected_groups]
        else:
            Absorbance_cols = self._get_avg_columns()
            
        df['Total_Absorbance'] = df[Absorbance_cols].sum(axis=1).astype(int)
        
        # Filter out zero Absorbance entries
        result_df = df[['unique ID', 'Total_Absorbance']]
        result_df = result_df[result_df['Total_Absorbance'] == 0]
        all_zero_list = list(result_df['unique ID'])
        peptides_df = df[~df['unique ID'].isin(all_zero_list)]

        # Process protein positions and create proteins DataFrame
        additional_columns = ['Master Protein Accessions', 'unique ID']
        selected_columns = additional_columns + Absorbance_cols
        
        peptides_df.loc[:, 'Master Protein Accessions'] = peptides_df['Master Protein Accessions']
        
        temp_df = peptides_df.copy()
        temp_df.loc[:, 'Protein_ID'] = temp_df['Master Protein Accessions']
        
        # Create proteins DataFrame with selected columns
        self.proteins_df = temp_df.groupby('Protein_ID').agg(
            {**{col: 'first' for col in ['Master Protein Accessions']},
            **{col: 'sum' for col in Absorbance_cols}}
        ).reset_index()
        
        # Calculate relative Absorbance for selected groups
        for col in Absorbance_cols:
            col_sum = self.proteins_df[col].sum()
            if col_sum > 0:  # Avoid division by zero
                self.proteins_df[f'Rel_{col}'] = (self.proteins_df[col] / col_sum) * 100
            else:
                self.proteins_df[f'Rel_{col}'] = 0
                
        # Create sum DataFrame for selected groups
        self.sum_df = pd.DataFrame({
            'Sample': Absorbance_cols,
            'Total_Sum': [self.proteins_df[col].sum() for col in Absorbance_cols]
        })
        
        # Add protein descriptions
        name_list = []
        for _, row in self.proteins_df.iterrows():
            if ',' in row['Protein_ID']:
                strrow = row['Protein_ID'].split(',')
                named_combo = self._fetch_protein_names('; '.join(strrow))
            else:
                named_combo = self._fetch_protein_names(row['Protein_ID'])
            name_list.append(named_combo)
        
        # Drop the 'Protein_ID' column
        self.proteins_df = self.proteins_df.drop(columns=['Protein_ID'])    
        
        self.proteins_df['Description'] = name_list
        self.proteins_df['Description'] = self.proteins_df['Description'].astype(str).str.replace(r"['\['\]]", "", regex=True)
        
        # Calculate average Absorbance for sorting using only selected groups
       
        # Calculate sum of all selected columns
        total_sum = self.proteins_df[Absorbance_cols].sum().sum()
        
        # Calculate row sums
        row_sums = self.proteins_df[Absorbance_cols].sum(axis=1)
        
        # Calculate relative percentage contribution
        self.proteins_df['avg_absorbance_all'] = (row_sums / total_sum * 100).round(2)
        
        # Sort proteins by abundance for consistent ordering
        self.proteins_df = self.proteins_df.sort_values('avg_absorbance_all', ascending=False)
                                
        # Create a dictionary to store the actual peptide counts per group
        self.peptide_count_totals = {}
        
        # Dictionary to store unique peptide counts per protein
        self.protein_peptide_counts = {}
        
        # Track which peptides belong to which proteins
        protein_to_peptides = defaultdict(set)
        
        # Track which peptides belong to which proteins in each group
        protein_to_group_peptides = defaultdict(lambda: defaultdict(set))
        
        # Determine counts based on merged_df and add to proteins_df
        if selected_groups and self.proteins_df is not None and df is not None:
            # Add count columns to the proteins_df (initialize with zeros)
            for group in selected_groups:
                count_col = f'Count_{group}'
                rel_count_col = f'Rel_Count_{group}'
                self.proteins_df[count_col] = 0
                self.proteins_df[rel_count_col] = 0  # Initialize relative count column
            
            # Create a mapping from accession to protein index in proteins_df
            accession_to_idx = {}
            accession_to_description = {}  # Map accessions to descriptions for counting
            for idx, row in self.proteins_df.iterrows():
                if 'Master Protein Accessions' in row and pd.notna(row['Master Protein Accessions']):
                    accession_to_idx[row['Master Protein Accessions']] = idx
                    accession_to_description[row['Master Protein Accessions']] = row['Description']
                elif 'Accession' in row and pd.notna(row['Accession']):
                    accession_to_idx[row['Accession']] = idx
                    accession_to_description[row['Accession']] = row['Description']
            
            # For each group, count peptides per protein
            for group in selected_groups:
                # Filter peptides that are present in this group
                group_peptides = df[df[f'Avg_{group}'] > 0]
                
                # Store the total number of peptides for this group
                self.peptide_count_totals[group] = len(group_peptides)
                
                # Track which peptides have already been counted
                counted_peptides = set()
                
                # Track warning stats
                peptides_with_no_accession = 0
                peptides_with_no_id = 0
                peptides_already_counted = 0
                peptides_with_multi_accessions = set()
                peptides_with_no_protein_match = 0
                
                # Count peptides for each protein
                for _, peptide in group_peptides.iterrows():
                    if 'Master Protein Accessions' not in peptide or pd.isna(peptide['Master Protein Accessions']):
                        peptides_with_no_accession += 1
                        continue
                        
                    # Get unique peptide ID to track counting
                    peptide_id = peptide.get('unique ID', None)
                    if peptide_id is None or pd.isna(peptide_id):
                        peptides_with_no_id += 1
                        continue  # Skip if no unique ID
                    
                    # Skip if we've already counted this peptide for this group
                    if peptide_id in counted_peptides:
                        peptides_already_counted += 1
                        continue
                    
                    accession = peptide['Master Protein Accessions']
                    found_match = False
                    
                    # Check if this peptide maps to multiple proteins
                    if ';' in accession:
                        peptides_with_multi_accessions.add(peptide_id)
                        accessions = [acc.strip() for acc in accession.split(';') if acc.strip()]
                        
                        # Only count for the first valid protein in the list
                        for acc in accessions:
                            if acc in accession_to_idx:
                                idx = accession_to_idx[acc]
                                count_col = f'Count_{group}'
                                self.proteins_df.at[idx, count_col] += 1
                                
                                # Add this peptide to the protein's set for protein-specific counting
                                protein_desc = accession_to_description.get(acc, acc)
                                protein_to_peptides[protein_desc].add(peptide_id)
                                protein_to_group_peptides[protein_desc][group].add(peptide_id)
                                
                                counted_peptides.add(peptide_id)  # Mark as counted
                                found_match = True
                                break  # Count only once
                    else:
                        # Handle direct match - only single protein
                        if accession in accession_to_idx:
                            idx = accession_to_idx[accession]
                            count_col = f'Count_{group}'
                            self.proteins_df.at[idx, count_col] += 1
                            
                            # Add this peptide to the protein's set for protein-specific counting
                            protein_desc = accession_to_description.get(accession, accession)
                            protein_to_peptides[protein_desc].add(peptide_id)
                            protein_to_group_peptides[protein_desc][group].add(peptide_id)
                            
                            counted_peptides.add(peptide_id)  # Mark as counted
                            found_match = True
                    
                    # Track peptides that didn't match any protein in our list
                    if not found_match:
                        peptides_with_no_protein_match += 1
                        
                # After counting all peptides for this group, calculate relative counts
                count_col = f'Count_{group}'
                rel_count_col = f'Rel_Count_{group}'
                total_value = self.peptide_count_totals[group]
                
                # Calculate relative counts as percentages of total peptides
                if total_value > 0:
                    for idx in range(len(self.proteins_df)):
                        protein_count = self.proteins_df.at[idx, count_col]
                        self.proteins_df.at[idx, rel_count_col] = (protein_count / total_value) * 100
                
                # Display warning about peptides mapping to multiple proteins
                warning_html = '<div style="color: orange; margin: 5px 0;"><b>Warning:</b> Peptide counting stats for group {0}:<br>'
                
                if peptides_with_no_accession > 0:
                    warning_html += f'• Skipped {peptides_with_no_accession} peptides with no accession<br>'
                    
                if peptides_with_no_id > 0:
                    warning_html += f'• Skipped {peptides_with_no_id} peptides with no unique ID<br>'
                    
                if peptides_already_counted > 0:
                    warning_html += f'• Skipped {peptides_already_counted} duplicate peptides (already counted)<br>'
                    
                if len(peptides_with_multi_accessions) > 0:
                    warning_html += f'• Found {len(peptides_with_multi_accessions)} peptides mapping to multiple proteins<br>'
                    warning_html += f'  (Each counted only once for the first matching protein)<br>'
                    
                if peptides_with_no_protein_match > 0:
                    warning_html += f'• {peptides_with_no_protein_match} peptides had no matching protein in the protein list<br>'
                    
                total_peptides = len(group_peptides)
                warning_html += f'• Total peptides processed: {total_peptides}, successfully counted: {len(counted_peptides)}'
                warning_html += '</div>'
                
                #display(HTML(warning_html.format(group)))

        # Calculate the number of unique peptides per protein
        for protein, peptides in protein_to_peptides.items():
            self.protein_peptide_counts[protein] = len(peptides)

        # Create a copy of the proteins DataFrame for protein sample distribution calculation
        working_df = self.proteins_df.copy()
        
        # Calculate protein distributions across samples (for both counts and absorbance)
        self.protein_sample_distribution = {}
        
        # Calculate data for major proteins (based on pro_list)
        major_proteins = []
        if hasattr(self, 'pro_list') and self.pro_list:
            major_proteins = self.pro_list
            
        # Add "Minor Proteins" data structures to hold aggregated values
        minor_proteins_data = {
            'counts': {group: 0 for group in selected_groups},
            'count_relative': {group: 0 for group in selected_groups},
            'absorbance': {group: 0 for group in selected_groups},
            'absorbance_relative': {group: 0 for group in selected_groups},
            'unique_peptide_count': 0,
            'total_value': 0,
            'total_absorbance': 0,
            'total_count': 0
        }
        
        # Counts to track minor proteins' peptides
        minor_proteins_peptides = set()
        
        # Process each protein
        for _, row in working_df.iterrows():
            protein_name = row['Description']
            
            # Skip if protein name is empty or NaN
            if pd.isna(protein_name) or not protein_name:
                continue
            
            # Initialize data structure for this protein
            protein_data = {
                'counts': {},
                'count_relative': {},
                'absorbance': {},
                'absorbance_relative': {},
                'unique_peptide_count': 0
            }
            
            # Get count values for each group
            count_values = {}
            absorbance_values = {}
            
            for group in selected_groups:
                # Get count values from proteins_df
                count_col = f'Count_{group}'
                if count_col in row:
                    count_values[group] = row[count_col]
                else:
                    count_values[group] = 0
                
                # Get absorbance values
                absorbance_col = f'Avg_{group}'
                if absorbance_col in row:
                    absorbance_values[group] = row[absorbance_col]
                else:
                    absorbance_values[group] = 0
            
            # Get the actual count of unique peptides for this protein (across all groups)
            if protein_name in protein_to_peptides:
                protein_data['unique_peptide_count'] = len(protein_to_peptides[protein_name])
            
            # Store the count and absorbance values
            protein_data['counts'] = count_values
            protein_data['absorbance'] = absorbance_values
            
            # Calculate totals as sums across groups
            protein_total_count = sum(count_values.values())
            protein_total_absorbance = sum(absorbance_values.values())
            
            protein_data['total_count'] = protein_total_count
            protein_data['total_absorbance'] = protein_total_absorbance
            
            # Calculate relative distributions
            # Count relative distribution - percentage of this protein's total count in each group
            if protein_total_count > 0:
                for group, count in count_values.items():
                    protein_data['count_relative'][group] = (count / protein_total_count) * 100
            else:
                for group in selected_groups:
                    protein_data['count_relative'][group] = 0
            
            # Absorbance relative distribution
            if protein_total_absorbance > 0:
                for group, absorbance in absorbance_values.items():
                    protein_data['absorbance_relative'][group] = (absorbance / protein_total_absorbance) * 100
            else:
                for group in selected_groups:
                    protein_data['absorbance_relative'][group] = 0
            
            # Add backward compatibility
            use_count = hasattr(self, 'abs_or_count') and ('count' in getattr(self, 'abs_or_count').value.lower() 
                                                        if hasattr(getattr(self, 'abs_or_count'), 'value') else True)
            
            if use_count:
                protein_data['total'] = protein_total_count
                protein_data['values'] = count_values
                protein_data['relative'] = protein_data['count_relative']
            else:
                protein_data['total'] = protein_total_absorbance
                protein_data['values'] = absorbance_values
                protein_data['relative'] = protein_data['absorbance_relative']
            
            # Check if this is a major or minor protein
            if major_proteins and protein_name not in major_proteins:
                # This is a minor protein - add its data to the minor proteins aggregated data
                for group in selected_groups:
                    minor_proteins_data['counts'][group] += count_values[group]
                    minor_proteins_data['absorbance'][group] += absorbance_values[group]
                
                # For minor proteins, track both the sum and the unique peptide count
                if protein_name in protein_to_peptides:
                    minor_proteins_peptides.update(protein_to_peptides[protein_name])
                
                minor_proteins_data['total_count'] += protein_total_count
                minor_proteins_data['total_absorbance'] += protein_total_absorbance
            else:
                # This is a major protein - store its individual data
                self.protein_sample_distribution[protein_name] = protein_data
        
        # Set unique peptide count for minor proteins
        minor_proteins_data['unique_peptide_count'] = len(minor_proteins_peptides)
        
        # Calculate relative distributions for minor proteins
        if minor_proteins_data['total_count'] > 0:
            for group in selected_groups:
                minor_proteins_data['count_relative'][group] = (minor_proteins_data['counts'][group] / minor_proteins_data['total_count'] * 100)
        
        if minor_proteins_data['total_absorbance'] > 0:
            for group in selected_groups:
                minor_proteins_data['absorbance_relative'][group] = (minor_proteins_data['absorbance'][group] / minor_proteins_data['total_absorbance'] * 100)
        
        # Add backward compatibility for minor proteins
        if use_count:
            minor_proteins_data['total'] = minor_proteins_data['total_count']
            minor_proteins_data['values'] = minor_proteins_data['counts']
            minor_proteins_data['relative'] = minor_proteins_data['count_relative']
        else:
            minor_proteins_data['total'] = minor_proteins_data['total_absorbance']
            minor_proteins_data['values'] = minor_proteins_data['absorbance']
            minor_proteins_data['relative'] = minor_proteins_data['absorbance_relative']
        
        # Add minor proteins to the distribution dictionary and peptide counts
        self.protein_sample_distribution['Minor Proteins'] = minor_proteins_data
        self.protein_peptide_counts['Minor Proteins'] = minor_proteins_data['unique_peptide_count']
        
        # Add a row for "Minor Proteins" to the proteins_df if not already present
        if 'Minor Proteins' not in self.proteins_df['Description'].values and major_proteins:
            minor_proteins_row = {'Description': 'Minor Proteins', 'Master Protein Accessions': 'Minor Proteins'}
            
            # Add counts and relative counts
            for group in selected_groups:
                count_col = f'Count_{group}'
                rel_count_col = f'Rel_Count_{group}'
                avg_col = f'Avg_{group}'
                rel_avg_col = f'Rel_{avg_col}'
                
                minor_proteins_row[count_col] = minor_proteins_data['counts'][group]
                minor_proteins_row[rel_count_col] = 0  # Will be recalculated below
                minor_proteins_row[avg_col] = minor_proteins_data['absorbance'][group]
                if rel_avg_col in self.proteins_df.columns:
                    # Calculate relative absorbance as percentage of total
                    total_absorbance = self.sum_df[self.sum_df['Sample'] == avg_col]['Total_Sum'].values[0]
                    if total_absorbance > 0:
                        minor_proteins_row[rel_avg_col] = (minor_proteins_data['absorbance'][group] / total_absorbance) * 100
                    else:
                        minor_proteins_row[rel_avg_col] = 0
            
            # Add the row
            self.proteins_df = pd.concat([self.proteins_df, pd.DataFrame([minor_proteins_row])], ignore_index=True)
            
            # Recalculate relative counts for all proteins
            for group in selected_groups:
                count_col = f'Count_{group}'
                rel_count_col = f'Rel_Count_{group}'
                total_count = self.peptide_count_totals[group]
                
                if total_count > 0:
                    for idx in range(len(self.proteins_df)):
                        protein_count = self.proteins_df.at[idx, count_col]
                        self.proteins_df.at[idx, rel_count_col] = (protein_count / total_count) * 100
        
        # Print debug info for key proteins
        #debug_proteins = ['Beta-casein', 'Alpha-S1-casein', 'Minor Proteins']
        #for protein in debug_proteins:
        #    if protein in self.protein_sample_distribution:
        #        print(f"\nProtein: {protein}")
        #        data = self.protein_sample_distribution[protein]
        #        print(f"Total count across all groups: {data['total_count']}")
        #        print(f"Unique peptide count: {data.get('unique_peptide_count', 'N/A')}")
        #        print(f"Total absorbance: {data['total_absorbance']:.2e}")
        #        print(f"Sample distribution (% of protein's total):")
        #        print(f"Count: {', '.join([f'{g}: {v:.1f}%' for g, v in data['count_relative'].items()])}")
        #        print(f"Absorbance: {', '.join([f'{g}: {v:.1f}%' for g, v in data['absorbance_relative'].items()])}")
        #        print(f"Sum of count percentages: {sum(data['count_relative'].values()):.1f}%")
                
        return True
    
    def _fetch_protein_names(self, accession_str):
        """
        Fetch protein names from the proteins dictionary.
        Returns a list of protein names, using the full protein name.
        """
        names = []
        for acc in accession_str.split('; '):
            if acc in self.data_transformer.proteins_dic:
                # Use the full protein name instead of splitting it
                name = self.data_transformer.proteins_dic[acc]['name']
                names.append(name)
            else:
                names.append(acc)
        return names

    def _get_color_sequence(self, n_colors):
        """Get color sequence based on selected scheme."""
        if n_colors <= 0:
            return []
        
        try:
            # Get the selected color scheme
            scheme = 'HSV'  # Default scheme
            if hasattr(self, 'color_scheme') and self.color_scheme.value:
                scheme = self.color_scheme.value
            
            # Skip header options that start with '---'
            if scheme.startswith('---'):
                scheme = 'HSV'  # Default to HSV if a header is selected
            
            # Handle special cases
            if scheme.lower() in ['rainbow', 'hsv']:
                return [f'hsl({h},70%,60%)' for h in np.linspace(0, 330, n_colors)]
            
            # Try qualitative color scales first (best for categorical data)
            color_sequence = getattr(px.colors.qualitative, scheme, None)
            if color_sequence is None:
                # Try sequential color scales
                color_sequence = getattr(px.colors.sequential, scheme, None)
            if color_sequence is None:
                # Try diverging color scales
                color_sequence = getattr(px.colors.diverging, scheme, None)
            if color_sequence is None:
                # Try cyclical color scales
                color_sequence = getattr(px.colors.cyclical, scheme, None)
            
            if color_sequence:
                if n_colors >= len(color_sequence):
                    # If we need more colors than available, interpolate
                    indices = np.linspace(0, len(color_sequence)-1, n_colors)
                    return [color_sequence[int(i)] for i in indices]
                else:
                    # If we need fewer colors, take a subset
                    indices = np.linspace(0, len(color_sequence)-1, n_colors, dtype=int)
                    return [color_sequence[i] for i in indices]
            
            # Default to HSV if no matching scheme found
            return [f'hsl({h},70%,60%)' for h in np.linspace(0, 330, n_colors)]
            
        except Exception as e:
            print(f"Error generating colors: {e}")
            # Fallback to HSV
            return [f'hsl({h},70%,60%)' for h in np.linspace(0, 330, n_colors)]

    def create_grouped_bar_plot(self, title, selected_groups, use_count=False):
        """Generate interactive Plotly grouped bar plots for proteins
        
        Uses pre-calculated data from process_data to create grouped bar plots.
        
        Args:
            title: Title for the plot
            selected_groups: Groups to include in the plot
            use_count: Whether to use counts instead of abundance values
            
        Returns:
            Plotly figure object
        """
        if self.proteins_df is None or len(self.proteins_df) == 0:
            print("No data available for plotting.")
            return None
        
        try:
            # Check if we're using relative metrics
            is_relative_metric = hasattr(self, 'metric_type') and 'relative' in self.metric_type.value.lower()
            
            # Create figure
            fig = go.Figure()
            
            # Determine orientation - sync with invert_plot for consistency
            if hasattr(self, 'invert_plot'):
                orientation = self.invert_plot.value
            else:
                orientation = 'By Sample'  # Default
            
            # Set up proteins to display
            if hasattr(self, 'pro_list') and self.pro_list:
                display_proteins = self.pro_list.copy()
                if hasattr(self, 'plot_minor_proteins') and self.plot_minor_proteins.value:
                    if 'Minor Proteins' in self.protein_sample_distribution:
                        display_proteins.append('Minor Proteins')
            else:
                # Fallback to top proteins
                proteins_to_show = int(self.minor_proteins_input.value) if hasattr(self, 'minor_proteins_input') else 10
                display_proteins = self.proteins_df['Description'].tolist()[:proteins_to_show]
            
            # Filter to only include proteins that have data
            display_proteins = [p for p in display_proteins if p in self.protein_sample_distribution]
            
            if not selected_groups or not display_proteins:
                print("No valid groups or proteins selected for plotting.")
                return None
            
            # Based on orientation, determine which will be the categories and which will be the bars
            if orientation == 'By Protein':
                # By Protein: Proteins on x-axis, samples as different colored bars
                categories = display_proteins
                bar_groups = selected_groups
                
                # Create consistent color mapping for samples
                color_sequence = self._get_color_sequence(len(selected_groups))
                color_mapping = {group: color_sequence[i] for i, group in enumerate(selected_groups)}
            else:  # 'By Sample'
                # By Sample: Samples on x-axis, proteins as different colored bars
                categories = selected_groups
                bar_groups = display_proteins
                color_sequence = self._get_color_sequence(len(display_proteins))
                color_mapping = {protein: color_sequence[i] for i, protein in enumerate(display_proteins)}
                # Special color for Minor Proteins
                if 'Minor Proteins' in bar_groups:
                    color_mapping['Minor Proteins'] = '#808080'  # Grey
            
            # Calculate bar positions
            n_bar_groups = len(bar_groups)
            bar_width = 0.8 / n_bar_groups  # Adjust total width of group
            
            # For each bar group, create a trace
            for idx, bar_group in enumerate(bar_groups):
                # Calculate x positions for this group's bars
                x_positions = [i + (idx - n_bar_groups/2 + 0.5) * bar_width for i in range(len(categories))]
                
                values = []
                hover_text = []
                
                for i, category in enumerate(categories):
                    if orientation == 'By Protein':
                        # By Protein: category = protein, bar_group = sample
                        protein = category
                        group = bar_group
                        
                        # Skip if protein doesn't have data
                        if protein not in self.protein_sample_distribution:
                            values.append(0)
                            hover_text.append(f"Protein: {protein}<br>Sample: {group}<br>Value: 0")
                            continue
                        
                        # Get data from protein_sample_distribution
                        protein_data = self.protein_sample_distribution[protein]
                        
                        # Get the appropriate values based on metric type
                        if use_count:
                            abs_value = protein_data['counts'].get(group, 0)
                            total_value = protein_data['total_count']
                            unique_count = protein_data['unique_peptide_count']
                            rel_percentage = protein_data['count_relative'].get(group, 0)
                        else:
                            abs_value = protein_data['absorbance'].get(group, 0)
                            total_value = protein_data['total_absorbance']
                            unique_count = protein_data['unique_peptide_count']
                            rel_percentage = protein_data['absorbance_relative'].get(group, 0)
                        
                        # Determine y-value based on metric type
                        if is_relative_metric:
                            # For relative metrics: use percentage contribution
                            y_value = rel_percentage
                            hover = (f"Protein: {protein}<br>" +
                                    f"Sample: {group}<br>" +
                                    f"Relative Contribution: {rel_percentage:.1f}%<br>" +
                                    f"{self.metric_name}: {abs_value:{self.num_format}}")
                        else:
                            # For absolute metrics: use actual count/abundance
                            y_value = abs_value
                            hover = (f"Protein: {protein}<br>" +
                                    f"Sample: {group}<br>" +
                                    f"{self.metric_name}: {abs_value:{self.num_format}}<br>" +
                                    f"Relative Contribution: {rel_percentage:.1f}%")
                        
                    else:  # 'By Sample'
                        # By Sample: category = sample, bar_group = protein
                        protein = bar_group
                        group = category
                        
                        # Skip if protein doesn't have data
                        if protein not in self.protein_sample_distribution:
                            values.append(0)
                            hover_text.append(f"Protein: {protein}<br>Sample: {group}<br>Value: 0")
                            continue
                        
                        # Get data from protein_sample_distribution
                        protein_data = self.protein_sample_distribution[protein]
                        
                        # Get the appropriate values based on metric type
                        if use_count:
                            abs_value = protein_data['counts'].get(group, 0)
                            total_count = self.peptide_count_totals.get(group, 0)
                            rel_percentage = (abs_value / total_count * 100) if total_count > 0 else 0
                            protein_specific_rel = protein_data['count_relative'].get(group, 0)
                        else:
                            abs_value = protein_data['absorbance'].get(group, 0)
                            # Get total absorbance for this group from sum_df
                            sample_col = f'Avg_{group}'
                            total_value = self.sum_df.loc[self.sum_df['Sample'] == sample_col, 'Total_Sum'].values[0] if sample_col in self.sum_df['Sample'].values else 0
                            rel_percentage = (abs_value / total_value * 100) if total_value > 0 else 0
                            protein_specific_rel = protein_data['absorbance_relative'].get(group, 0)
                        
                        # Determine y-value based on metric type
                        if is_relative_metric:
                            # For relative metrics: use percentage of sample total
                            y_value = rel_percentage
                            hover = (f"Protein: {protein}<br>" +
                                    f"Sample: {group}<br>" +
                                    f"Relative Contribution: {rel_percentage:.1f}%<br>" +
                                    f"{self.metric_name}: {abs_value:{self.num_format}}")
                        else:
                            # For absolute metrics: use actual count/abundance
                            y_value = abs_value
                            hover = (f"Protein: {protein}<br>" +
                                    f"Sample: {group}<br>" +
                                    f"{self.metric_name}: {abs_value:{self.num_format}}<br>" +
                                    f"Relative Contribution: {rel_percentage:.1f}%")
                    
                    values.append(y_value)
                    hover_text.append(hover)
                
                # Only add trace if we have valid values
                if any(v > 0 for v in values):
                    fig.add_trace(go.Bar(
                        name=bar_group,
                        x=x_positions,
                        y=values,
                        width=bar_width * 0.9,  # Slight gap between bars
                        marker_color=color_mapping.get(bar_group, 'gray'),
                        hovertext=hover_text,
                        hoverinfo='text'
                    ))
            
            # Determine plot title and axis labels
            plot_title = title or f"Protein Distribution - {orientation} ({self.metric_name})"
            
            # Determine y-axis title based on metric type
            if use_count:
                metric_base = "Peptide Count"
            else:
                metric_base = "Abundance"
                
            if is_relative_metric:
                yaxis_title = f"Relative {metric_base} (%)"
            else:
                yaxis_title = f"Absolute {metric_base}"
            
            # Determine x-axis title based on orientation
            xaxis_title = 'Proteins' if orientation == 'By Protein' else 'Sample Categories'
            
            # Determine legend title based on orientation
            legend_title = 'Samples' if orientation == 'By Protein' else 'Protein Origins'
            
            # Update titles from widgets if available
            if hasattr(self, 'legend_widget') and self.legend_widget.value:
                legend_title = self.legend_widget.value
            
            if hasattr(self, 'xlabel_widget') and self.xlabel_widget.value:
                xaxis_title = self.xlabel_widget.value

            if hasattr(self, 'ylabel_widget') and self.ylabel_widget.value:
                yaxis_title = self.ylabel_widget.value

            # Update layout
            fig.update_layout(
                title={
                    'text': plot_title,
                    'y': .975,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {'size': 18, 'color': 'black'}
                },
                xaxis_title=xaxis_title,
                yaxis_title=yaxis_title,
                legend_title=legend_title,
                legend={'yanchor': "top", 'y': 1.0, 'xanchor': "left", 'x': 1.05, 'traceorder': 'normal', 'font': {'size': 12, 'color': 'black'}},
                showlegend=True,
                template='plotly_white',
                height=750,
                width=1100,
                margin=dict(t=100, l=100, r=200),
                hoverlabel=dict(
                    bgcolor="white",
                    font_size=12,
                    font_family="Arial"
                ),
                barmode='group',
                xaxis=dict(
                    showline=True,
                    linewidth=1,
                    linecolor='black',
                    mirror=False
                ),
                yaxis=dict(
                    showline=True,
                    linewidth=1,
                    linecolor='black',
                    mirror=False
                )
            )
            
            # Update axis properties
            fig.update_xaxes(
                ticktext=categories,
                tickvals=list(range(len(categories))),
                tickangle=45,
                title_font={"size": 18},
                tickfont={"size": 16},
                tickfont_color="black",
                title_font_color="black",
            )
            
            # Set y-axis format based on plot type
            if use_count and not is_relative_metric:
                # Absolute count
                fig.update_yaxes(
                    type='linear',
                    tickformat=",d",  # Format with commas for thousands
                    title_font={"size": 18},
                    tickfont={"size": 16},
                    tickfont_color="black",
                    title_font_color="black",
                    gridcolor="lightgray",
                    showgrid=True,
                    zeroline=False,
                )
            elif not use_count and not is_relative_metric:
                # Absolute abundance
                fig.update_yaxes(
                    type='log',
                    exponentformat='E',
                    showexponent='all',
                    title_font={"size": 18},
                    tickfont={"size": 16},
                    tickfont_color="black",
                    title_font_color="black",
                    gridcolor="lightgray",
                    showgrid=True,
                    zeroline=False,
                )
            else:
                # Relative metrics (both count and abundance)
                fig.update_yaxes(
                    type='linear',
                    range=[0, 100],
                    title_font={"size": 18},
                    tickfont={"size": 16},
                    tickfont_color="black",
                    title_font_color="black",
                    gridcolor="lightgray",
                    showgrid=True,
                    zeroline=False,
                )
                
            # Mark generation as complete
            self.state_manager.generate_completed()
            self.export_button.disabled = False
            self.download_plot_button.disabled = False
            
            return fig
            
        except Exception as e:
            print(f"Error creating grouped bar plot: {str(e)}")
            import traceback
            traceback.print_exc()
            return None
   
    def generate_download_link(self, content, filename, filetype='text/csv'):
        """Generate a download link for any content"""
        if isinstance(content, pd.DataFrame):
            if filetype == 'text/csv':
                content = content.to_csv(index=False)
            else:
                content = content.to_csv(index=True)
        if isinstance(content, str):
            content = content.encode()
        b64 = base64.b64encode(content).decode()
        return f"""
            <a id="download_link" href="data:{filetype};base64,{b64}" 
               download="{filename}"
               style="display: none;">
                Download {filename}
            </a>
            <script>
                document.getElementById('download_link').click();
            </script>
            """
    
    def _on_download_plot_click(self, b):
        """Handle plot download button click with automatic download"""
        if self.current_fig is not None:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            plot_filename = f'protein_plot_{timestamp}.html'
            
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(self.generate_download_link(
                    self.current_fig.to_html(),
                    plot_filename,
                    'text/html'
                )))
        else:
            print("Please generate a plot first.")
    
    def display(self):
        """Display the protein analysis interface"""
        display(self.widget_box)

    def _on_export_button_click(self, b):
        """Handle data export with automatic download"""
        if self.proteins_df is not None:
            timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
            data_filename = f'protein_absorbance_analysis_{timestamp}.csv'
            
            with self.export_output:
                self.export_output.clear_output(wait=True)
                display(HTML(self.generate_download_link(
                    self.proteins_df,
                    data_filename,
                    'text/csv'
                )))
        else:
            print("Please generate the analysis first.")

    def _on_plot_button_click(self, b):
        if self.current_fig is not None:
            self.state_manager.generate_completed()
            self.export_button.disabled = False
            self.download_plot_button.disabled = False
        with self.plot_output:
            self.plot_output.clear_output(wait=True)
            
            if not self.group_select.value:
                print("Please select at least one group to plot.")
                return
            
            selected_groups = list(self.group_select.value)
            if self.process_data(selected_groups):

                # Check if using count metric
                use_count = False
                self.metric_name = "Abundance"
                if hasattr(self, 'abs_or_count'):
                    if self.abs_or_count.value == 'Count':
                        use_count = True
                        self.metric_name = "Peptide Count"
                        self.value_prefix = "Count_"
                        self.rel_prefix = "Rel_Count_"
                        self.num_format = ",.0f"  # Integer format for counts
                        
                    else: #abundance
                        self.metric_name = "Abundance"
                        self.value_prefix = "Avg_"
                        self.rel_prefix = "Rel_Avg_"
                        self.num_format = ",.2e"  # Scientific notation for abundance
            
                # Create mapping from sample name to column names
                self.value_cols = {var: f'{self.value_prefix}{var}' for var in selected_groups}
                self.rel_cols = {var: f'{self.rel_prefix}{var}' for var in selected_groups}
                    
                # Get the selected plot type (Bar or Pie)
                if hasattr(self, 'plot_type'):
                    plot_type = self.plot_type.value
                else:
                    plot_type = 'Stacked Bar Plots'  # Default to bar plot if widget doesn't exist
                
                # Create and display the appropriate plot based on selection and metric
                if plot_type == 'Stacked Bar Plots':
                    # Create and store bar plot (existing functionality)
                    title = self.title_widget.value or f'Protein Distribution Analysis ({self.metric_name})'
                    
                    # Modified plot_stacked_bar_scaled to use count columns if needed
                    self.current_fig = self.plot_stacked_bar_scaled(
                        title=title,
                        selected_groups=selected_groups,
                        use_count=use_count
                    )
                    if self.current_fig is not None:
                        self.current_fig.show()
                
                elif plot_type == "Grouped Bar Plots":
                    # Get the group column - you might want to add a dropdown for this
                    title = self.title_widget.value or f'Protein Distribution Analysis ({self.metric_name})'

                    self.current_fig = self.create_grouped_bar_plot(
                            title=title,
                            selected_groups=selected_groups,
                            use_count=use_count
                        )
                    if self.current_fig is not None:
                        self.current_fig.show()
                elif plot_type == 'Pie Charts':  # Pie Chart
                    # Create and store pie plot
                    try:
                        # Call the pie chart function with count flag
                        orientation = self.invert_plot.value
                        title = self.title_widget.value or f"Protein Distribution - {orientation} ({self.metric_name})"

                        self.current_fig = self.create_pie_charts(
                            selected_groups=selected_groups, 
                            title=title,
                            use_count=use_count
                        )
                        
                        if self.current_fig is not None:
                            self.current_fig.show()
                        else:
                            print("Error: Could not generate pie charts.")
                    except Exception as e:
                        print(f"Error creating pie charts: {str(e)}")
                        import traceback
                        traceback.print_exc()
            else:
                print("Please upload all required files first.")
                print("Error creating plot. Please check your data.")
                 
    def create_pie_charts(self, selected_groups, title, use_count=False):
        """Create pie charts for protein data with pre-calculated counts or abundance"""
        try:
            # Check if we have data
            if not hasattr(self, 'proteins_df') or self.proteins_df is None or self.proteins_df.empty:
                print("No protein data available to plot")
                return None
                        
            # Determine orientation - sync with invert_plot for consistency
            if hasattr(self, 'invert_plot'):
                orientation = self.invert_plot.value
            else:
                orientation = 'By Sample'  # Default
            
            
            # Create figure and subplots with maximum 3 columns
            if orientation == 'By Sample':
                # One pie chart per sample
                # Create a working copy of the DataFrame named scaled_df
                scaled_df = self.proteins_df.copy()
                
                # Get the list of groups/samples to plot
                if hasattr(self, 'selected_groups'):
                    selected_groups = self.selected_groups
                else:
                    # Fallback if selected_groups is not available
                    selected_groups = list(self.value_cols.keys()) if isinstance(self.value_cols, dict) else [col.replace(self.value_prefix, '') for col in self.value_cols]
                
                # Map group names to actual column names
                sample_columns = []
                sample_names = []
                
                if isinstance(self.value_cols, dict):
                    # If value_cols is a dictionary (group -> column)
                    for group in selected_groups:
                        if group in self.value_cols:
                            sample_columns.append(self.value_cols[group])
                            sample_names.append(group)
                else:
                    # If value_cols is a list of columns
                    sample_columns = self.value_cols
                    sample_names = [col.replace(self.value_prefix, '') for col in sample_columns]
                
                # Ensure we have data after mapping
                if not sample_columns:
                    print("No samples available for plotting")
                    return None
                
                # Filter to only selected proteins (including minor proteins if enabled)
                if hasattr(self, 'pro_list') and self.pro_list:
                    display_proteins = self.pro_list.copy()
                    if hasattr(self, 'plot_minor_proteins') and self.plot_minor_proteins.value:
                        if 'Minor Proteins' in scaled_df['Description'].values:
                            display_proteins.append('Minor Proteins')
                    
                    scaled_df = scaled_df[scaled_df['Description'].isin(display_proteins)]
                
                # Debug info
                print(f"Sample columns to plot: {sample_columns}")
                print(f"Sample names for display: {sample_names}")
                
                # Check if columns exist in DataFrame
                for col in sample_columns:
                    if col not in scaled_df.columns:
                        print(f"Warning: Column '{col}' not found in DataFrame. Available columns: {scaled_df.columns.tolist()}")
                
                # Calculate grid layout
                num_samples = len(sample_columns)
                num_cols = min(3, num_samples)  # Maximum 3 columns
                num_rows = (num_samples + num_cols - 1) // num_cols  # Ceiling division
                
                # Create figure with grid layout - using sample_names as subplot_titles
                fig = make_subplots(
                    rows=num_rows, 
                    cols=num_cols,
                    specs=[[{'type': 'pie'} for _ in range(num_cols)] for _ in range(num_rows)],
                    subplot_titles=sample_names
                )
                
                # Get unique proteins for coloring
                unique_proteins = scaled_df['Description'].unique().tolist()
                if 'Minor Proteins' in unique_proteins:
                    unique_proteins.remove('Minor Proteins')
                
                # Use the existing color sequence function for proteins
                protein_colors = self._get_color_sequence(len(unique_proteins))
                
                # Create a color map, setting Minor Proteins to grey
                color_map = {protein: color for protein, color in zip(unique_proteins, protein_colors)}
                if 'Minor Proteins' in scaled_df['Description'].values:
                    color_map['Minor Proteins'] = '#808080'  # Grey color for minor proteins
                
                # First pie chart will set the legend for all
                first_chart = True
                
                # Create a pie chart for each sample
                for i, col_name in enumerate(sample_columns):
                    # Skip if column doesn't exist in DataFrame
                    if col_name not in scaled_df.columns:
                        continue
                        
                    # Calculate which row and column this chart belongs in
                    row_idx = i // num_cols + 1  # 1-based indexing for plotly
                    col_idx = i % num_cols + 1   # 1-based indexing for plotly
                    
                    sample_name = sample_names[i]
                    
                    # Get data for this sample
                    try:
                        sample_data = scaled_df[['Description', col_name]].copy()
                        
                        # Filter out zero values
                        sample_data = sample_data[sample_data[col_name] > 0]
                        
                        # Skip if no data
                        if sample_data.empty:
                            continue
                        
                        # Sort by value but ensure Minor Proteins is at the end
                        if 'Minor Proteins' in sample_data['Description'].values:
                            # Extract Minor Proteins row
                            minor_row = sample_data[sample_data['Description'] == 'Minor Proteins']
                            # Get the rest sorted by value
                            other_rows = sample_data[sample_data['Description'] != 'Minor Proteins']
                            other_rows = other_rows.sort_values(by=col_name, ascending=False)
                            # Combine with Minor Proteins at the end
                            sample_data = pd.concat([other_rows, minor_row], ignore_index=True)
                        else:
                            sample_data = sample_data.sort_values(by=col_name, ascending=False)
                        
                        # Get colors for the current sample's proteins
                        colors = [color_map.get(protein, '#CCCCCC') for protein in sample_data['Description']]
                        
                        # Create the pie chart
                        fig.add_trace(
                            go.Pie(
                                labels=sample_data['Description'],
                                values=sample_data[col_name],
                                name=sample_name,
                                marker_colors=colors,
                                textposition='inside',
                                textinfo='percent',
                                hovertemplate=(
                                    f"Protein: %{{label}}<br>"
                                    f"{self.metric_name}: %{{value:{self.num_format}}}<br>"
                                    f"Percentage: %{{percent}}<br>"
                                    f"Sample: {sample_name}<br>"
                                    f"<extra></extra>"
                                ),
                                hole=0.3,
                                showlegend=first_chart  # Only show legend for the first chart
                            ),
                            row=row_idx, col=col_idx
                        )
                        
                        # After first chart, don't show labels in legend again
                        first_chart = False
                        
                    except Exception as e:
                        print(f"Error creating pie chart for {sample_name}: {str(e)}")
                        continue

           # pie plot
            else:  # 'By Protein'
                # One pie chart for each protein showing distribution across samples
                # Convert dict_keys to a list to make it subscriptable
                unique_proteins = list(self.protein_sample_distribution.keys())
                
                # Filter to only include proteins we want to plot
                if hasattr(self, 'pro_list') and self.pro_list:
                    # Include both selected proteins and Minor Proteins if enabled
                    filtered_proteins = []
                    for protein in self.pro_list:
                        if protein in self.protein_sample_distribution:
                            filtered_proteins.append(protein)
                    
                    if self.plot_minor_proteins.value and 'Minor Proteins' in self.protein_sample_distribution:
                        filtered_proteins.append('Minor Proteins')
                        
                    unique_proteins = filtered_proteins
                
                # Ensure we have proteins to plot
                if not unique_proteins:
                    print("No proteins available with distribution data")
                    return None
                    
                num_proteins = len(unique_proteins)
                
                num_cols = min(3, num_proteins)  # Maximum 3 columns
                num_rows = (num_proteins + num_cols - 1) // num_cols  # Ceiling division
                
                # Create figure with grid layout
                fig = make_subplots(
                    rows=num_rows, 
                    cols=num_cols,
                    specs=[[{'type': 'pie'} for _ in range(num_cols)] for _ in range(num_rows)],
                    subplot_titles=unique_proteins
                )
                
                # Use the existing color sequence function for samples
                sample_colors = self._get_color_sequence(len(selected_groups))
                
                # Create a color map for samples
                color_map = {group: color for group, color in zip(selected_groups, sample_colors)}
                
                # First pie chart will set the legend for all
                first_chart = True
                
                # Create a pie chart for each protein
                for i, protein in enumerate(unique_proteins):
                    # Calculate which row and column this chart belongs in
                    row_idx = i // num_cols + 1  # 1-based indexing for plotly
                    col_idx = i % num_cols + 1   # 1-based indexing for plotly
                    
                    # Get the pre-calculated data from the dictionary
                    protein_data = self.protein_sample_distribution[protein]
                    
                    # Choose the appropriate data based on metric type
                    if use_count:
                        values_dict = protein_data['counts']
                        total = protein_data['total_count']
                    else:
                        values_dict = protein_data['absorbance'] 
                        total = protein_data['total_absorbance']
                        
                    # Get values and labels, filtering out zero values
                    values = []
                    labels = []
                    colors_list = []
                    
                    for group in selected_groups:
                        value = values_dict.get(group, 0)
                        if value > 0:  # Only include non-zero values
                            values.append(value)
                            labels.append(group)
                            colors_list.append(color_map[group])
                    
                    # Skip if no non-zero values
                    if not values:
                        continue
                        
                    # Create the pie chart
                    fig.add_trace(
                        go.Pie(
                            labels=labels,
                            values=values,
                            name=protein,
                            marker_colors=colors_list,
                            textposition='inside',
                            textinfo='percent',
                            hovertemplate=(
                                f"Sample: %{{label}}<br>"
                                f"{self.metric_name}: %{{value:{self.num_format}}}<br>"
                                f"Percentage: %{{percent}}<br>"
                                f"Total {self.metric_name}: {total:{self.num_format}}<br>"
                                f"<extra></extra>"
                            ),
                            hole=0.3,
                            showlegend=first_chart  # Only show legend for the first chart
                        ),
                        row=row_idx, col=col_idx
                    )
                    
                    # After first chart, don't show labels in legend again
                    first_chart = False
            
            # Update layout with adjusted height for multiple rows
            legend_title = self.legend_widget.value or ('Samples' if self.invert_plot.value == 'By Protein' else 'Protein Origins')

            
            fig.update_layout(
                height=500 * num_rows,  # Scale height based on number of rows
                width=min(1400, 450 * num_cols),  # Adjust width for maximum 3 columns
                title_text=title,
                title={
                    'y': 0.98,
                    'x': 0.5,
                    'xanchor': 'center',
                    'yanchor': 'top',
                    'font': {"size": 20, 'color': 'black'}
                },
                showlegend=True,  # Show the legend
                legend={
                    'title': legend_title,
                    'yanchor': "top",
                    'y': 0.99,
                    'xanchor': "left",
                    'x': 1.02,
                    'font': {"size": 12},
                    #'bgcolor': 'rgba(255, 255, 255, 0.8)',
                    #'bordercolor': 'rgba(0, 0, 0, 0.5)',
                    #'borderwidth': 1
                },
                margin=dict(t=100, b=50, l=50, r=150),  # Increased right margin for legend
                paper_bgcolor='rgba(255,255,255,1)',
                plot_bgcolor='rgba(255,255,255,1)',
                font=dict(
                    family="Arial, sans-serif",
                    size=14,
                    color="black"
                )
            )
            
            return fig
            
        except Exception as e:
            print(f"Error creating pie charts: {str(e)}")
            import traceback
            traceback.print_exc()
            return None   

    def plot_stacked_bar_scaled(self, title, selected_groups, use_count=False):
        if self.proteins_df is None or self.sum_df is None:
            return None
                    
        # Create figure object before adding traces
        fig = go.Figure()
        
        # Check if we're using relative metrics
        is_relative_metric = hasattr(self, 'metric_type') and 'relative' in self.metric_type.value.lower()
        

        if self.invert_plot.value == 'By Sample':

            scaled_df = self.proteins_df.copy()
        
            # Calculate total values for each sample (needed for hover info)
            total_absorbance = {}
            for group in selected_groups:
                col = self.value_cols[group]
                total_absorbance[group] = self.proteins_df[col].sum()
            
            # For relative metric, ensure all relative columns are properly calculated
            if is_relative_metric: # 'relative'
                for group in selected_groups:
                    value_col = self.value_cols[group]
                    rel_col = self.rel_cols[group]
                    
                    # If relative column doesn't exist, calculate it
                    if rel_col not in scaled_df.columns:
                        total = scaled_df[value_col].sum()
                        if total > 0:
                            scaled_df[rel_col] = scaled_df[value_col] / total * 100
                        else:
                            scaled_df[rel_col] = 0
            
            # Calculate total sums for each group first (for absolute values)
            total_sums = {}
            for group in selected_groups:
                if use_count:
                    # For count values, use the pre-calculated totals from process_data
                    if hasattr(self, 'peptide_count_totals') and group in self.peptide_count_totals:
                        total_sum = self.peptide_count_totals[group]
                    else:
                        # Fallback: directly sum the Count_ columns
                        count_col = f'Count_{group}'
                        if count_col in self.proteins_df.columns:
                            total_sum = self.proteins_df[count_col].sum()
                        else:
                            total_sum = 0
                            print(f"Warning: Count column {count_col} not found in proteins_df")
                else: # abundance
                    # For abundance values, use the existing logic
                    sample_key = self.value_cols[group]
                    if sample_key in self.sum_df['Sample'].values:
                        # Direct match
                        total_sum = self.sum_df.loc[self.sum_df['Sample'] == sample_key, 'Total_Sum'].values[0]
                    else:
                        # Calculate from proteins_df
                        total_sum = self.proteins_df[sample_key].sum()
                
                # Store with clean group name
                total_sums[group] = total_sum
                

            # Create a new DataFrame for the Minor Proteins
            minor_proteins_df = pd.DataFrame()

            # Calculate minor proteins data for each sample
            for group in selected_groups:
                # Get the value and relative columns
                value_col = self.value_cols[group]
                rel_col = self.rel_cols[group]
                
                # Sum values for proteins not in pro_list
                minor_proteins_value = scaled_df[~scaled_df['Description'].isin(self.pro_list)][value_col].sum()
                
                # Calculate relative value for minor proteins
                if is_relative_metric:
                    # When using relative metrics, calculate percentage directly
                    total_value = scaled_df[value_col].sum()
                    minor_proteins_rel_value = minor_proteins_value / total_value * 100 if total_value > 0 else 0
                else:
                    # For absolute metrics, use the pre-calculated relative column if it exists
                    if rel_col in scaled_df.columns:
                        minor_proteins_rel_value = scaled_df[~scaled_df['Description'].isin(self.pro_list)][rel_col].sum()
                    else:
                        # If it doesn't exist, calculate it directly
                        minor_proteins_rel_value = 0
                
                # Add to the minor proteins DataFrame
                if len(minor_proteins_df) == 0:
                    minor_proteins_df = pd.DataFrame({
                        'Description': ['Minor Proteins'],
                        'Master Protein Accessions': ['Minor Proteins'],
                        rel_col: [minor_proteins_rel_value],
                        value_col: [minor_proteins_value]
                    })
                else:
                    minor_proteins_df[rel_col] = minor_proteins_rel_value
                    minor_proteins_df[value_col] = minor_proteins_value

            # Filter scaled_df to only include proteins in pro_list
            scaled_df = scaled_df[scaled_df['Description'].isin(self.pro_list)]
            
            # Sort proteins_df based on pro_list
            description_order = {desc: i for i, desc in enumerate(self.pro_list)}
            scaled_df['Order'] = scaled_df['Description'].map(description_order)
            scaled_df = scaled_df.sort_values(by='Order').reset_index(drop=True)
                                
            # Now only append if the checkbox is checked
            if self.plot_minor_proteins.value:
                # Append minor proteins to the end of the main dataframe
                scaled_df = pd.concat([scaled_df, minor_proteins_df], ignore_index=True)


            # Get colors based on selected color scheme
            colors = self._get_color_sequence(len(self.pro_list))
            # Add gray color for Minor Proteins
            colors.append('#808080')  # Gray color for Minor Proteins
            
            # For each sample, calculate the relative proportions and scale by the actual count
            for group in selected_groups:
                # Get the total count for this sample
                sample_total = total_sums[group]
                
                # Create a DataFrame with just this sample's data for scaling
                sample_data = scaled_df.copy()
                
                # Get column to use (rel column for relative metrics, value column for absolute)  
                col = self.rel_cols[group] if is_relative_metric else self.value_cols[group]
                
                # Calculate the raw percentage for each protein in this sample
                if not is_relative_metric: # absolute
                    # For absolute metrics, calculate percentage
                    sample_col_total = sample_data[col].sum()
                    if sample_col_total > 0:
                        sample_data[f'percentage_{group}'] = sample_data[col] / sample_col_total * 100
                    else:
                        sample_data[f'percentage_{group}'] = 0
                else: # relative
                    # For relative metrics, use the relative column directly
                    sample_data[f'percentage_{group}'] = sample_data[col]
                
                # Calculate the scaled height for each protein in this sample
                if use_count:
                    # For count data, scale by the total count
                    sample_data[f'scaled_{group}'] = sample_data[f'percentage_{group}'] * sample_total / 100
                    if is_relative_metric:
                        sample_data[f'scaled_{group}'] = sample_data[f'percentage_{group}']
                else:
                    # For abundance data, use the original values
                    sample_data[f'scaled_{group}'] = sample_data[col]
                
                # Filter out zero values for plotting
                for idx, row in sample_data.iterrows():
                    protein_description = row['Description']
                    if protein_description in self.pro_list or protein_description == 'Minor Proteins':
                        # Use gray for Minor Proteins, otherwise use the color from the sequence
                        if protein_description == 'Minor Proteins':
                            color = '#808080'  # Gray color
                        else:
                            color = colors[self.pro_list.index(protein_description)]
                        
                        # Get the scaled value for this protein in this sample
                        value = row[f'scaled_{group}']
                        
                        # Create hover text
                        abs_value = row[f'{self.value_prefix}{group}']
                        rel_value = row[f'{self.rel_prefix}{group}'] 
                        

                        hover_text = (
                            f"Protein: {row['Master Protein Accessions']}<br>" +
                            f"Description: {row['Description']}<br>" +
                            f"Sample: {group}<br>" +
                            f"Relative {self.metric_name}: {rel_value:.2f}%<br>" +
                            f"Absolute {self.metric_name}: {abs_value:{self.num_format}}<br>"
                        )
                        
                        # Add trace for this protein in this sample
                        fig.add_trace(go.Bar(
                            name=protein_description,
                            x=[group],  # Just this sample
                            y=[value],  # Scaled value
                            marker_color=color,
                            hovertext=[hover_text],
                            hoverinfo='text',
                            showlegend=(group == selected_groups[0])  # Only show in legend for first sample
                        ))
                                
        else:  # 'By Protein'
            # Inverted plotting logic (proteins on x-axis)
            # Get colors based on selected color scheme
            colors = self._get_color_sequence(len(selected_groups))

            # Define protein_totals dictionary to store totals for each protein
            protein_totals = {}

            # For each protein, get data from the protein_sample_distribution dictionary
            for protein in self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else []):
                # Get total from protein_sample_distribution if available
                protein_data = self.protein_sample_distribution[protein]
                if use_count:
                    protein_totals[protein] = protein_data['unique_peptide_count']
                else:
                    protein_totals[protein] = protein_data['total']

            # Now plot each protein with data from protein_sample_distribution
            for protein in self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else []):
                # Skip if protein not found in data
                if protein not in self.protein_sample_distribution.keys():
                    continue
                
                # Check if we have data in the protein_sample_distribution dictionary
                protein_distribution_available = (hasattr(self, 'protein_sample_distribution') and 
                                                    protein in self.protein_sample_distribution)
                
                if not protein_distribution_available:
                    print(f"Warning: No distribution data available for {protein}")
                    continue
                
                # Get the pre-calculated data from the dictionary
                protein_data = self.protein_sample_distribution[protein]
                
                # Get the appropriate values based on metric type
                sample_values = protein_data['values']
                sample_percentages = protein_data['relative']
                total_value = protein_data['total']
                unique_peptide_count = protein_data['unique_peptide_count']
                
                # Only proceed if we have a valid total
                if total_value <= 0:
                    continue
                
                # Debug output for key proteins
                #if protein in ['Beta-casein', 'Alpha-S1-casein', 'Alpha-S2-casein']:
                #    print(f"Protein: {protein}")
                #    print(f"Total {self.metric_name}: {total_value}")
                #    print(f"Sample percentages: {sample_percentages}")
                
                # For each sample, add a bar segment
                for i, group in enumerate(selected_groups):
                    # Skip if no contribution
                    if group not in sample_values or sample_values[group] <= 0:
                        continue
                    
                    abs_value = sample_values[group]  # Actual count/abundance in this sample
                    rel_percentage = sample_percentages.get(group, 0)  # Sample's percentage contribution
                    
                    # Calculate y-value based on metric type
                    if is_relative_metric: #Relative
                        # For relative metrics: always plot relative percentages (0-100 scale)
                        y_value = rel_percentage

                    else: #Absolute
                        # For absolute metrics: scale each segment to maintain the correct total height
                        # This ensures segments add up to the total count/abundance
                        if use_count:
                            y_value = unique_peptide_count * (rel_percentage / 100)

                        else:
                            y_value = total_value * (rel_percentage / 100)

                    # Format for hover
                    if use_count:
                        self.num_format = ",.0f"  # Integer format for counts
                        abs_count_label = "Count"
                    else:
                        self.num_format = ".2e"   # Scientific notation for abundance
                        abs_count_label = "Abundance"
                    
                    # Create hover text with correct information
                    hover_text = (
                        f"Protein: {protein}<br>" +
                        f"Sample: {group}<br>" +
                        f"Sample's contribution to protein: {rel_percentage:.2f}%<br>" +
                        f"{abs_count_label} in sample: {abs_value:{self.num_format}}<br>" 
                        #f"Total protein {abs_count_label}: {total_value:{self.num_format}}"
                    )
                    
                    # Add trace for this sample in this protein
                    fig.add_trace(go.Bar(
                        name=group,
                        x=[protein],
                        y=[y_value],
                        marker_color=colors[i],
                        hovertext=[hover_text],
                        hoverinfo='text',
                        showlegend=(protein == (self.pro_list[0] if self.pro_list else 'Minor Proteins'))
                    ))

            # Set Y-axis range to 0-100 for relative metrics
            if is_relative_metric:
                fig.update_layout(yaxis_range=[0, 100])
        
        # Get custom labels
        x_label = self.xlabel_widget.value or ('Proteins' if self.invert_plot.value == 'By Protein' else 'Samples')
        y_label = self.ylabel_widget.value or (f"Relative {self.metric_name} within {'protein' if self.invert_plot.value == 'By Protein' else 'sample'} (%)" if is_relative_metric else self.metric_name)
        plot_title = title
        legend_title = self.legend_widget.value or ('Samples' if self.invert_plot.value == 'By Protein' else 'Protein Origins')

        fig.update_layout(
            barmode='stack',
            title={
                'text': plot_title,
                'y': 0.95,
                'x': 0.5,
                'xanchor': 'center',
                'yanchor': 'top',
                'font': {"size": 18, 'color': 'black'}
            },
            xaxis_title=x_label,
            yaxis_title=y_label,
            yaxis=dict(
                showline=True,
                gridcolor='lightgray',
                showgrid=True,
                showticklabels=True,
                linewidth=1,
                linecolor='black',
                mirror=False,
                zeroline=False,  # Don't show zero line
                range=[0, 100] if is_relative_metric else None  # Set range to [0,100] for relative metrics
            ),
            xaxis=dict(
                showline=True,
                linewidth=1,
                linecolor='black',
                mirror=False,
                tickangle=-90 if self.invert_plot.value == 'By Sample' else 45  # Adjust tick angle based on orientation
            ),
            legend_title=legend_title,
            legend={
                'yanchor': "top",
                'y': 0.95,
                'xanchor': "left",
                'x': 1.05,
                'traceorder': 'normal',
                'font': {"size": 16, 'color': 'black'},
                'bgcolor': 'rgba(255, 255, 255, 0.9)'
            },
            showlegend=True,
            template='plotly_white',
            height=820,
            width=1200,
            margin=dict(
                t=100,
                l=100,
                r=100,
                b=100
            ),
            hoverlabel=dict(
                bgcolor="white",
                font_size=14,
                font_family="Arial"
            ))
        
        fig.update_xaxes(
            tickangle=45,
            title_font={"size": 18},
            tickfont={"size": 16},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title                
        )
        
        # Update Y axis formatting based on metric
        if is_relative_metric:
            tick_format = ".1f"  # Format as percentage with one decimal place for relative metrics
            showticklabels_tf = True
        else:
            if use_count:
                tick_format = ""  # Regular integers for counts
            else:
                tick_format = ".1e"  # Scientific notation for abundance
            showticklabels_tf = False
        fig.update_yaxes(
            title_font={"size": 18},
            tickfont={"size": 16},
            tickfont_color="black",  # Black tick labels
            title_font_color="black",  # Black axis title
            gridcolor="lightgray",  # Light gray grid lines
            showgrid=True,  # Show grid lines
            zeroline=False,  # Hide zero line
            exponentformat='E',
            showexponent='all',
            tickformat=tick_format,
            showticklabels=showticklabels_tf
        )
        
        # Always add scatter trace for totals, but calculate differently based on orientation
        if not is_relative_metric:  # Only show totals for absolute metrics
            if self.invert_plot.value == 'By Sample':
                # Sample-wise totals calculation
                # Format based on metric
                if use_count:
                    text_format = [f"{int(total_sums[group])}" for group in selected_groups]
                else: #abundance
                    text_format = [f"{total_sums[group]:.2e}" for group in selected_groups]
                    
                fig.add_trace(go.Scatter(
                    x=selected_groups,
                    y=[total_sums[group] for group in selected_groups],
                    mode='text',
                    text=text_format,
                    textposition='top center',
                    textfont=dict(size=12, color='black'),
                    showlegend=True,
                    name=f'Show Total {self.metric_name}',
                    hoverinfo='none',
                    texttemplate='%{text}'
                ))
            else: #By Protein
                # Protein-wise totals
                # Format for display based on metric
                if use_count:
                    text_format = [f"{int(protein_totals.get(protein, 0))}" for protein in self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else [])]
                else: #abundance
                    # For abundance, calculate the sum across all samples
                    text_format = [f"{protein_totals.get(protein, 0):.2e}" for protein in self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else [])]
                y_values = [protein_totals.get(protein, 0) for protein in self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else [])]
                
                # Add the protein totals trace
                fig.add_trace(go.Scatter(
                    x=self.pro_list + (['Minor Proteins'] if self.plot_minor_proteins.value else []),
                    y=y_values,
                    mode='text',
                    text=text_format,
                    textposition='top center',
                    textfont=dict(size=12, color='black'),
                    showlegend=True,
                    name=f'Show Total {self.metric_name}',
                    hoverinfo='none',
                    texttemplate='%{text}'
                ))       
                    
        # Mark generation as complete
        self.state_manager.generate_completed()
        self.export_button.disabled = False
        self.download_plot_button.disabled = False
        
        return fig
    

In [23]:

# Initialize the interface
data_transformer = DataTransformation()
data_transformer.setup_data_loading_ui()

# Create protein plotter
protein_plotter = ProteinPlotter(data_transformer)

protein_plotter.display()

Output(layout=Layout(margin='0 0 0 20px', width='300px'))

In [24]:
protein_plotter.value_cols

AttributeError: 'ProteinPlotter' object has no attribute 'value_cols'